In [ ]:
# 获取股票行情数据

In [ ]:
from typing import List
from typing import Dict

# 日期工具
from datetime import datetime, timedelta
from xtquant import xtdata

In [ ]:
# 获取沪深A股中所有成分股的列表
sector_name: str = '沪深A股'
stock_list_in_sector: List[str] = xtdata.get_stock_list_in_sector(sector_name)
print(f'获取板块成分股列表{len(stock_list_in_sector)}成功：{stock_list_in_sector}')

In [ ]:
# 测试下载000001.SZ的日 k 行情数据并获取打印
stock_code: str = '000001.SZ'
print(f'开始下载股票{stock_code}的行情数据')
# 获取五年前的日期,yyyyMMdd格式
one_years_ago = (datetime.now() - timedelta(days=1 * 365)).strftime('%Y%m%d')
print(f'一年前的日期为：{one_years_ago}')
# 获取今天日期
today = datetime.now().strftime('%Y%m%d')
xtdata.download_history_data(stock_code = stock_code, period = '1d', start_time='20250101', end_time='20250606', incrementally = False)
print(f'股票{stock_code}的行情数据下载完成')

In [ ]:
# 获取下载的行情数据
stock_data = xtdata.get_market_data(
    field_list=['time', 'open', 'high', 'low', 'close', 'volume', 'amount', 'settle', 'openInterest'],
    stock_list=[stock_code],
    period='1d',
    start_time=one_years_ago,
    end_time=today,
    count=-1,
    fill_data=False
)
print(f'获取股票 {stock_code} 的行情数据成功：{stock_data}')
# 查看返回数据的格式
print(f'获取股票 {stock_code} 的行情数据格式：{type(stock_data)}')

In [ ]:
# 获取指定stock_list的行情数据函数
def get_stock_data(stock_list: List[str], start_time: str, end_time: str) -> Dict:
    """
    获取指定股票列表的行情数据
    :param stock_list: 股票列表
    :param start_time: 开始时间，格式为 'yyyyMMdd'
    :param end_time: 结束时间，格式为 'yyyyMMdd'
    """
    # 获取每只股票的日K 行情数据
    stock_data = xtdata.get_market_data(
        field_list=['time', 'open', 'high', 'low', 'close', 'volume', 'amount', 'settle', 'openInterest'],
        stock_list=stock_list,
        period='1d',
        start_time=start_time,
        end_time=end_time,
        count=1,
        dividend_type='cash_dividend',
        fill_data=False
    )
    print(f'获取股票 {stock_list} 的行情数据{len(stock_data)}成功：{stock_data}')
    return stock_data

In [ ]:
code = '000001.SZ'
data = xtdata.get_market_data(['close'], [code], period='1d', start_time='20250423')
print('一次性取数据', data)

In [ ]:
stock_data = xtdata.get_market_data(
        field_list=['time', 'open', 'high', 'low', 'close', 'volume', 'amount', 'settle', 'openInterest'],
        stock_list=['000001.SZ'],
        period='1d',
        start_time='20250101',
        end_time='20250901',
        count=1,
        fill_data=False
    )
print(f'获取股票的行情数据成功：{stock_data}')

In [ ]:
# 获取000001.SZ的20250101-20250606的数据
stock_data = get_stock_data(
    stock_list=['000001.SZ'],
    start_time='20250101',
    end_time='20250601'
)

In [ ]:
# 获取沪深A股中所有成分股的过去 5 年的日K 行情数据
"""
"time"                #时间戳
"open"                #开盘价
"high"                #最高价
"low"                 #最低价
"close"               #收盘价
"volume"              #成交量
"amount"              #成交额
"settle"              #今结算
"openInterest"        #持仓量
"""
# 遍历成分股列表
for stock in stock_list_in_sector:
    # 获取五年前的日期,yyyyMMdd格式
    five_years_ago = (datetime.now() - timedelta(days=5 * 365)).strftime('%Y%m%d')
    print(f'五年前的日期为：{five_years_ago}')
    # 获取今天日期
    today = datetime.now().strftime('%Y%m%d')
    # 获取每只股票的过去 5 年的日K 行情数据
    stock_data = xtdata.get_market_data(
        field_list=['time','open','high','low','close','volume','amount','settle','openInterest'],
        stock_list=stock,
        period='1d',
        start_time=five_years_ago,
        end_time=today,
        count=-1,
        dividend_type = 'cash_dividend', # todo: 待确认
        fill_data = False
        )
    # 打印每只股票的过去 5 年的日K 行情数据
    print(f'获取股票 {stock} 的过去 5 年的日K 行情数据成功：{stock_data}')